In [120]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

pd.options.display.max_rows = None

two_months_ago = datetime.today() - timedelta(weeks=4)

## Reformatação Teste

In [110]:
produtos = requests.get('http://127.0.0.1:5000/produtos').json()
pd.DataFrame(produtos)

,codigo,estoque,estoque_minimo,id,marca,nome,slug,unidade
0,ALCAQP,9.27,8.62,623449734,Porcão,ALCATRA COM QUEIJO,Alcatra,kg
1,ALMOBARB,0,5.00,623449795,Recheados Brasil,ALMÔNDEGA BACON,Almôndega,kg
2,ALMOQRB,0,4.50,623449782,Recheados Brasil,ALMÔNDEGA QUEIJO,Almôndega,kg
3,ANCHALI,0,0.00,778208483,Alimentar,ANCHO ALIMENTAR,Ancho,kg
4,ANCHDIPDI,0,0.00,646749750,Di Prima,ANCHO DI PRIMA,Ancho,kg
5,ANCHMINERV,0,0.00,729652533,Minerva,ANCHO MINERVA,Ancho,kg
6,ANCHMOSTKMA,14.73,4.00,638403161,Montana Steakhouse,ANCHO MONTANA STEAKHOUSE,Ancho,kg
7,ASARECHRB,7,5.00,623449824,Recheados Brasil,ASINHA RECHEADA,Asinha,un
8,ASSTIRARG,3.15,2.00,623449837,ARG,ASSADO DE TIRAS ARG,Assado,kg
9,BACNFATDNT,0,2.00,661843960,,BACON FATIADO,Bacon,un


In [37]:
produtos

[{'codigo': 'ALCAQP',
  'estoque': 9.27,
  'estoque_minimo': 8.62,
  'id': '623449734',
  'marca': 'Porcão',
  'nome': 'ALCATRA COM QUEIJO',
  'slug': 'Alcatra',
  'unidade': 'kg'},
 {'codigo': 'ALMOBARB',
  'estoque': 0,
  'estoque_minimo': 5.0,
  'id': '623449795',
  'marca': 'Recheados Brasil',
  'nome': 'ALMÔNDEGA BACON',
  'slug': 'Almôndega',
  'unidade': 'kg'},
 {'codigo': 'ALMOQRB',
  'estoque': 0,
  'estoque_minimo': 4.5,
  'id': '623449782',
  'marca': 'Recheados Brasil',
  'nome': 'ALMÔNDEGA QUEIJO',
  'slug': 'Almôndega',
  'unidade': 'kg'},
 {'codigo': 'ANCHALI',
  'estoque': 0,
  'estoque_minimo': 0.0,
  'id': '778208483',
  'marca': 'Alimentar',
  'nome': 'ANCHO ALIMENTAR',
  'slug': 'Ancho',
  'unidade': 'kg'},
 {'codigo': 'ANCHDIPDI',
  'estoque': 0,
  'estoque_minimo': 0.0,
  'id': '646749750',
  'marca': 'Di Prima',
  'nome': 'ANCHO DI PRIMA',
  'slug': 'Ancho',
  'unidade': 'kg'},
 {'codigo': 'ANCHMINERV',
  'estoque': 0,
  'estoque_minimo': 0.0,
  'id': '729652533'

## Test Data Reformat

In [368]:
pedido = produtos[5:30]
pedido = pd.DataFrame(pedido)
pedido['cotacao'] = 14.52
pedido['quantidade'] = 4

pedido.to_json('produtos.json', orient='records')

In [369]:
pedido.drop['estoque', ]

,codigo,estoque,estoque_minimo,id,marca,nome,slug,unidade,cotacao,quantidade
0,ANCHMINERV,0.00,0.0,729652533,Minerva,ANCHO MINERVA,Ancho,kg,14.52,4
1,ANCHMOSTKMA,14.73,4.0,638403161,Montana Steakhouse,ANCHO MONTANA STEAKHOUSE,Ancho,kg,14.52,4
2,ASARECHRB,7.00,5.0,623449824,Recheados Brasil,ASINHA RECHEADA,Asinha,un,14.52,4
3,ASSTIRARG,3.15,2.0,623449837,ARG,ASSADO DE TIRAS ARG,Assado,kg,14.52,4
4,BACNFATDNT,0.00,2.0,661843960,,BACON FATIADO,Bacon,un,14.52,4
5,BUTCHPORC,0.00,0.0,649329262,Porcão,BUTCHER STEAK PORCAO,Butcher,kg,14.52,4
6,CARPACCP,0.00,0.0,733149637,Cara Preta,CARPACCIO CARA PRETA,Carpaccio,un,14.52,4
7,CARREPLE,0.00,1.0,659216777,1953,CARRÉ CORDEIRO 1953,Carré Cordeiro,kg,14.52,4
8,CHRZEST92,9.97,4.0,623449886,Estância 92,CHORIZO ESTÂNCIA 92,Chorizo,kg,14.52,4
9,CORAGAT,5.00,6.0,623449902,Jerônimos,CORAÇÃO,Coração,kg,14.52,4


## Get Pedido

In [398]:
pedido = pd.read_json('produtos.json', orient='records')
pedido['total'] = pedido['cotacao'] * pedido['quantidade']
pedido = pedido[pedido['marca'] != '']
pedido = pd.pivot_table(pedido, values=['total'], index='marca', aggfunc='sum').reset_index()

pedido

,marca,total
0,1953,58.08
1,ARG,58.08
2,Alimentar,58.08
3,Bassi,58.08
4,Burg,58.08
5,Cara Preta,232.32
6,Dia a Dia,58.08
7,Ester de Lima,58.08
8,Estância 92,116.16
9,Jerônimos,116.16


## Situação Atual

In [99]:
# vendas
vendas = list()
data = {
    'formato': 'json',
    'token': '2b6fc7102240cedcc9166c43921ea73eea82b876',
    'dataInicial': two_months_ago.strftime('%d/%m/%Y'),
    'dataFinal': datetime.today().strftime('%d/%m/%Y'),
    'pagina': 1
}
while True:
    try:

        url = 'https://api.tiny.com.br/api2/pedidos.pesquisa.php'
        vendas += [x['pedido'] for x in requests.post(url, data).json()['retorno']['pedidos']]
        data['pagina'] += 1
    except:
        break
vendas = pd.DataFrame(vendas)


In [121]:
# faturamento
faturamento_estimado = vendas['valor'].sum()/4
faturamento_estimado

29980.185

## Agendamento

In [443]:
# prazos
prazos = pd.read_csv('prazos.csv')
prazos.columns = ['marca',1,2,3]
prazos['n_parcelas'] = 3 - prazos.T.isna().sum()
prazos = prazos.merge(pedido)

prazos

,marca,1,2,3,n_parcelas,total
0,Alimentar,14.0,NaN,NaN,1,58.08
1,Cara Preta,7.0,14.0,NaN,2,232.32
2,Ester de Lima,7.0,14.0,NaN,2,58.08
3,Estância 92,14.0,21.0,28.0,3,116.16
4,Jerônimos,7.0,14.0,NaN,2,116.16
5,Porcão,7.0,14.0,NaN,2,58.08
6,Porcão,7.0,14.0,NaN,2,58.08
7,Minerva,14.0,21.0,28.0,3,174.24
8,Recheados Brasil,7.0,14.0,NaN,2,116.16


In [444]:
# agenda
agenda = prazos.copy()
agenda
for parcela in [1,2,3]:
    agenda[parcela] = [datetime.today() + timedelta(days=x) if x > 0 else None for x in prazos[parcela]]
agenda['valor'] = agenda['total']/agenda['n_parcelas']
agenda = agenda.melt(id_vars=['marca', 'valor'], value_vars=[1,2,3], value_name='data', var_name='parcela').dropna()
agenda

,marca,valor,parcela,data
0,Alimentar,58.08,1,2022-06-12 17:22:01.727563
1,Cara Preta,116.16,1,2022-06-05 17:22:01.727563
2,Ester de Lima,29.04,1,2022-06-05 17:22:01.727563
3,Estância 92,38.72,1,2022-06-12 17:22:01.727563
4,Jerônimos,58.08,1,2022-06-05 17:22:01.727563
5,Porcão,29.04,1,2022-06-05 17:22:01.727563
6,Porcão,29.04,1,2022-06-05 17:22:01.727563
7,Minerva,58.08,1,2022-06-12 17:22:01.727563
8,Recheados Brasil,58.08,1,2022-06-05 17:22:01.727563
10,Cara Preta,116.16,2,2022-06-12 17:22:01.727563


In [307]:
# ultima parcela
ultima_parcela = agenda['data'].max()
ultima_parcela

Timestamp('2022-07-28 15:34:02.450877')

,total
marca,
1953,58.08
ARG,58.08
Alimentar,58.08
Bassi,58.08
Burg,58.08
Cara Preta,232.32
Dia a Dia,58.08
Ester de Lima,58.08
Estância 92,116.16
